In [28]:
## ライブラリのimport
# !pip install mip 
from mip import Model, maximize, minimize, xsum
import numpy as np
from numpy import pi
from numpy.random import rand, randn
from numpy.linalg import solve, det
from scipy.linalg import sqrtm
import math
import statistics

## 定数の設定
p = 2 # 行列の次元
N = 500 # データ数
delta = 10.0
epsilon = 1.0
lam = 0.02
h = 4.0 #U(-h,h)

## Function to calculate entropy regularized Wasserstein distance
def reg_wasserstein(x, y):
    M_lam = sqrtm(lam * lam * np.eye(p) + sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2.0 * np.trace(M_lam) - 2.0 * lam * math.log(det(M_lam - lam * np.eye(p)),math.e) - 2.0 * lam * p * math.log(2.0 * lam, math.e) - 4.0 * lam * p * math.log(2.0 * math.pi, math.e) - 2.0 * lam * p + 2.0 * lam * math.log(det(x @ y),math.e) + 4.0 * lam * p * (math.log(2.0 * math.pi, math.e) + 1.0)
    return np.sqrt(res)

# 各シードでのカバリングナンバーを格納
cn_reg = []

for seed in range(10):    
    ## 乱数の生成と距離グラフの生成
        
    ### regularized
    P = []
    nbh_rw = [[] for _ in range(N)]


    i = 0
    np.random.seed(seed)
    while i < N:
        a = np.array([[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h - h],[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h * h]])
        a = a.reshape((2,2)) 
        v = a @ a.T
        # Skip if not within B(I,delta)
        if reg_wasserstein(np.eye(p),v) > delta:
            continue
        
        for j in range(0,i):
            rw_dist = reg_wasserstein(P[j],v)
            if rw_dist < epsilon:
                nbh_rw[i].append(j)
                nbh_rw[j].append(i)
                
        # print(i)    
        nbh_rw[i].append(i)
                
        i += 1
        P.append(v)
        
    ## Covering number
    ### ragularized
    m = Model()  # 数理モデル
    x = []
    # 変数
    for i in range(N):
        s = "x"+str(i)
        x.append(m.add_var(s, lb=0, var_type="I"))

    # 目的関数
    z = x[0]
    for i in range(1,N):
        z += x[i]

    m.objective = minimize(z)
    # m.objective = maximize(100 * x + 100 * y)
    # 制約条件
    for i in range(N):
        b = 1
        for j in nbh_rw[i]:
            b += x[j]
        m += b >= 2
        
    m.optimize()  # ソルバーの実行
    print(m.objective.x)
    cn_reg.append(m.objective.x)

print("reg_wass lam =" + str(lam) +": mean:"   + str(statistics.mean(cn_reg)) + " std: " + str(statistics.pstdev(cn_reg)))

203.0
202.0
189.0
200.0
195.0
202.0
210.0
207.0
202.0
196.0
reg_wass lam =0.02: mean:200.6 std: 5.7306195127577615


In [29]:
print("reg_wass lam =" + str(lam) +": mean:"   + str(statistics.mean(cn_reg)) + " std: " + str(statistics.pstdev(cn_reg)))

reg_wass lam =0.02: mean:200.6 std: 5.7306195127577615


In [30]:
# a = np.array([[1,0],[0,1]])
# a = a.reshape((2,2)) 
# x = reg_wasserstein(a,a)
# print(x)